In [1]:
import time
import asyncio
import pymongo
import json
from pymongo import MongoClient
from indy import anoncreds, did, ledger, pool, wallet, IndyError

async def run():
    print("Getting started -> started")

    # Set protocol version 2 to work with Indy Node 1.4
    await pool.set_protocol_version(2)

    pool_ = {
        'name': 'pool1',
        'config': json.dumps({"genesis_txn": '/home/indy/sandbox/pool_transactions_genesis'})
    }
    print("Open Pool Ledger: {}".format(pool_['name']))

    try:
        await pool.create_pool_ledger_config(pool_['name'], pool_['config'])
    except IndyError as ex:
        if ex.error_code == 306:
            pass

    pool_['handle'] = await pool.open_pool_ledger(pool_['name'], None)

    client = MongoClient(host='steward_db', port=27017, username='root', password='pass', authSource="admin")
    db = client["steward_db"]


    # Putting pool config in Database
    #db.steward_tb.find_one_and_update(pool_);
    #db.steward_tb.find_one_and_update({"name": "pool1"}, { "$set": { "value": pool_} }, upsert=True);
    db.steward_tb.insert_one(pool_);
    #pool_1 = db.steward_tb.find_one({"name": "pool1"})
    #pool_1['handle'] = await pool.open_pool_ledger(pool_['name'], None)

    steward = {
        'name': "Sovrin Steward",
        'wallet_config': json.dumps({'id': 'sovrin_steward_wallet'}),
        'wallet_credentials': json.dumps({'key': 'steward_wallet_key'}),
        'pool': pool_['handle'],
        'seed': '000000000000000000000000Steward1'
    }

    await create_wallet(steward)

    print("\"Sovrin Steward\" -> Create and store in Wallet DID from seed")
    steward['did_info'] = json.dumps({'seed': steward['seed']})
    steward['did'], steward['key'] = await did.create_and_store_my_did(steward['wallet'], steward['did_info'])

    db.steward_tb.insert_one(steward);
    # steward1 = db.steward_tb.find_one({"name": "Sovrin Steward"})
    # print(steward1)

    # print("==============================")

    # supplier = {
    #     'name': 'Supplier',
    #     'wallet_config': json.dumps({'id': 'supplier_wallet'}),
    #     'wallet_credentials': json.dumps({'key': 'supplier_wallet_key'}),
    #     'pool': pool_1['handle'],
    #     'role': 'TRUST_ANCHOR'
    # }
    # await create_wallet(supplier)

    # (supplier['did'], supplier['key']) = await did.create_and_store_my_did(supplier['wallet'], "{}")

    # db.steward_tb.insert_one(supplier);
    # supplier1 = db.steward_tb.find_one({"name": "Supplier"})
    # print(steward1)
    # print(supplier1)
    # await getting_verinym(steward1, supplier1)
    # print(supplier1)

    print("Getting started -> done")

   


async def create_wallet(identity):
    print("\"{}\" -> Create wallet".format(identity['name']))
    try:
        await wallet.create_wallet(identity['wallet_config'], identity['wallet_credentials'])
    except IndyError as ex:
        if ex.error_code == 306:
            pass
    identity['wallet'] = await wallet.open_wallet(identity['wallet_config'], identity['wallet_credentials'])

# async def getting_verinym(from_, to):
#     from_['info'] = {
#         'did': to['did'],
#         'verkey': to['key'],
#         'role': to['role'] or None
#     }
#     print(from_['wallet'])
#     print("In getting verinym")
#     await send_nym(from_['pool'], from_['wallet'], from_['did'], from_['info']['did'],
#                    from_['info']['verkey'], from_['info']['role'])

# async def send_nym(pool_handle, wallet_handle, _did, new_did, new_key, role):
#     nym_request = await ledger.build_nym_request(_did, new_did, new_key, None, role)
#     await ledger.sign_and_submit_request(pool_handle, wallet_handle, _did, nym_request)
#     print("In send nym")



 # print("==============================")
    # print("=== Supplier Credential Schemas Setup ==")
    # print("------------------------------")

    # print("\"Supplier\" -> Create \"Supplier Attestation\" Schema")
    # supplier_certificate = {
    #     'name': 'Supplier-Certificate',
    #     'version': '0.2',
    #     'attributes': ['first_name', 'last_name', 'salary', 'employee_status', 'experience']
    # }

    # await supplier_cred_schema(supplier,supplier_certificate)

    # print("==============================")
    # print("=== Supplier Credential Definition Setup ==")
    # print("------------------------------")

   
    # supplier_certificate_cred_def = {
    #     'tag': 'TAG1',
    #     'type': 'CL',
    #     'config': {"support_revocation": False}
    # }

    # await supplier_cred_definition(supplier, supplier_certificate_cred_def)
    # print(supplier)

    # print("============Device1 Registration=========")

    # device1 = {
    #     'name': 'Device1',
    #     'wallet_config': json.dumps({'id': 'device_wallet'}),
    #     'wallet_credentials': json.dumps({'key': 'device_wallet_key'}),
    #     'pool': pool_['handle'],
    # }

    # await device1_registration(device1)

    # print("Getting started done")


# async def device1_registration(device1):
#     await create_wallet(device1)
#     (device1['did'], device1['key']) = await did.create_and_store_my_did(device1['wallet'], "{}")

# async def supplier_cred_scheme(supplier,supplier_certificate):
#     (supplier['supplier_certificate_schema_id'], supplier['supplier_certificate_schema']) = \
#         await anoncreds.issuer_create_schema(supplier['did'], supplier_certificate['name'], supplier_certificate['version'],
#                                              json.dumps(supplier_certificate['attributes']))
#     supplier_certificate_schema_id = supplier['supplier_certificate_schema_id']

#     print("\"Supplier\" -> Send \"Supplier-Certificate\" Schema to Ledger")
#     await send_schema(supplier['pool'], supplier['wallet'], supplier['did'], supplier['supplier_certificate_schema'])

# async def supplier_cred_definition(supplier, supplier_certificate_cred_def):
#     print("\"Supplier\" -> Get from Ledger \"Supplier-Certificate\" Schema")

#     (supplier['supplier_certificate_schema_id'], supplier['supplier_certificate_schema']) = \
#         await get_schema(supplier['pool'], supplier['did'], supplier_certificate_schema_id)

#     (supplier['supplier_certificate_cred_def_id'], supplier['supplier_certificate_cred_def']) = \
#         await anoncreds.issuer_create_and_store_credential_def(supplier['wallet'], supplier['did'],
#                                                                supplier['supplier_certificate_schema'],
#                                                                supplier_certificate_cred_def['tag'],
#                                                                supplier_certificate_cred_def['type'],
#                                                                json.dumps(supplier_certificate_cred_def['config']))

    # print("\"Supplier\" -> Send \"Supplier Supplier-Certificate\" Credential Definition to Ledger")
    # await send_cred_def(supplier['pool'], supplier['wallet'], supplier['did'], supplier['supplier_certificate_cred_def'])



if __name__ == '__main__':
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    await run()
    time.sleep(1)  # FIXME waiting for libindy thread complete


Getting started -> started
Open Pool Ledger: pool1
"Sovrin Steward" -> Create wallet
"Sovrin Steward" -> Create and store in Wallet DID from seed
Getting started -> done
